In [1]:
import numpy as np
import pandas as pd
import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
import warnings
import gc
import sys
import pickle
# from sklearn.preprocessing import OneHotEncoder

In [2]:
# for train , some rows have to be dropped
# drop test set with no weather available

In [3]:
building = pd.read_csv('files/building_metadata2.csv')
# weather_train = pd.read_csv('files/weather_train_agg.csv')
# train = pd.read_csv('files/train.csv')
# dump_name = 'train_1209'
weather_train = pd.read_csv('files/weather_test_agg.csv')
train = pd.read_csv('files/test.csv')
dump_name = 'test_1209'

In [4]:
train.shape

(41697600, 4)

In [5]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
#             print("******************************")
#             print("Column: ",col)
#             print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
#             print("dtype after: ",props[col].dtype)
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [6]:
building, NAlist = reduce_mem_usage(building)

Memory usage of properties dataframe is : 0.0664520263671875  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  0.023613929748535156  MB
This is  35.535304247990815 % of the initial size


In [7]:
weather_train, NAlist = reduce_mem_usage(weather_train)

Memory usage of properties dataframe is : 40.18885040283203  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  15.59969425201416  MB
This is  38.81597531566835 % of the initial size


In [8]:
train, NAlist = reduce_mem_usage(train)

Memory usage of properties dataframe is : 1272.5098876953125  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  596.4890747070312  MB
This is  46.87500509621608 % of the initial size


In [9]:
train = train.merge(building, on='building_id', how='left')

In [10]:
del building
gc.collect()

20

In [11]:
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [12]:
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')

In [13]:
del weather_train
gc.collect();

In [14]:
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

weekday               199029
hour                  199029
month                 199029
year                  199029
air_temperature       199029
cloud_coverage        199029
dew_temperature       199029
precip_depth_1_hr     199029
sea_level_pressure    199029
wind_direction        199029
wind_speed            199029
humidity              199029
apparent_temp         199029
day                   199029
min_temperature       199029
max_temperature       199029
mean_temperature      199029
dtype: int64


In [15]:
# only for train
# https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type
    
# Removing weired data on site_id 0

# As you can see above, this data looks weired until May 20. It is reported in this discussion by @barnwellguy that All electricity meter is 0 until May 20 for site_id == 0. I will remove these data from training data.

# It corresponds to building_id <= 104.


# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113254#latest-667876

# 363 consume energy before built

In [16]:
# train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [17]:
# train = train.query('not (building_id == 363)')

In [18]:
train.shape

(41697600, 26)

In [19]:
train.drop(['day','year'],axis=1,inplace=True)
# train.drop(['timestamp','day','year'],axis=1,inplace=True)

In [20]:
train.shape

(41697600, 24)

In [21]:
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

weekday               199029
hour                  199029
month                 199029
air_temperature       199029
cloud_coverage        199029
dew_temperature       199029
precip_depth_1_hr     199029
sea_level_pressure    199029
wind_direction        199029
wind_speed            199029
humidity              199029
apparent_temp         199029
min_temperature       199029
max_temperature       199029
mean_temperature      199029
dtype: int64


In [ ]:
# drop test set with no weather available

In [22]:
train = train[pd.notnull(train['air_temperature'])]

In [23]:
train.shape

(41498571, 24)

In [24]:
train['wind_direction'][train['wind_direction'] == 360] = 0
train['wind_direction'] = pd.Series(train['wind_direction']/45, dtype='uint8')

In [25]:
display(train)

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,floor_count,age,weekday,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,humidity,apparent_temp,min_temperature,max_temperature,mean_temperature
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,0,9,6.0,...,11.7,0.0,1021.400024,2,3.6,67.5,14.010,12.2,27.200001,18.987499
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,0,13,6.0,...,11.7,0.0,1021.400024,2,3.6,67.5,14.010,12.2,27.200001,18.987499
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,0,26,6.0,...,11.7,0.0,1021.400024,2,3.6,67.5,14.010,12.2,27.200001,18.987499
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,1,15,6.0,...,11.7,0.0,1021.400024,2,3.6,67.5,14.010,12.2,27.200001,18.987499
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,6,42,6.0,...,11.7,0.0,1021.400024,2,3.6,67.5,14.010,12.2,27.200001,18.987499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41498566,41498566,1369,2,2018-12-31 23:00:00,15,Entertainment/public assembly,53194,5,64,0.0,...,2.2,0.0,1014.700012,3,5.1,92.5,-1.217,-4.4,3.300000,-0.575000
41498567,41498567,1402,0,2018-12-31 23:00:00,15,Lodging/residential,133594,11,56,0.0,...,2.2,0.0,1014.700012,3,5.1,92.5,-1.217,-4.4,3.300000,-0.575000
41498568,41498568,1417,2,2018-12-31 23:00:00,15,Lodging/residential,40451,3,42,0.0,...,2.2,0.0,1014.700012,3,5.1,92.5,-1.217,-4.4,3.300000,-0.575000
41498569,41498569,1431,2,2018-12-31 23:00:00,15,Public services,111360,9,17,0.0,...,2.2,0.0,1014.700012,3,5.1,92.5,-1.217,-4.4,3.300000,-0.575000


In [28]:
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [29]:
train, NAlist = reduce_mem_usage(train)

Memory usage of properties dataframe is : 4392.949467658997  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  3324.3941917419434  MB
This is  75.67567567567568 % of the initial size


In [30]:
with open(dump_name, 'wb') as f:
    pickle.dump(train, f)